# Scan conversion

#### Objective

Transform a series of line into an image that geometrically makes sense:

![](images/7317421464689659964.jpg)

#### Litterature

* Interesting http://www.k-wave.org/documentation/scanConversion.php 
* https://www.researchgate.net/publication/265425106_Signal_Processing_Overview_of_Ultrasound_Systems_for_Medical_Imaging/figures?lo=1
* https://www.researchgate.net/publication/259614810_Programmable_ultrasound_scan_conversion_on_a_media-processor-based_system